In [97]:
import pandas as pd
import numpy as np
import json
import csv
import pymongo
import re
import io
import glob
import time
from datetime import datetime
import requests
from io import BytesIO
import webbrowser

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from tkinter import filedialog
import tkinter as tk


In [101]:
window = tk.Tk()
window.title('The Tool v0.42b')
window.geometry('400x210')
window.resizable(0,0)

# style
style1 = ttk.Style()
style1.theme_use('winnative')
style1.configure("Yell.TLabel", foreground='#939597', background='#F5DF4D', font='helvetica 24')

# content, frame and label
content = ttk.Frame(window, padding=(3,3,3,3))
frame_r = ttk.Frame(content, borderwidth=5, relief='ridge')
frame_l = ttk.Frame(content, borderwidth=5, relief='ridge')
namelbl = ttk.Label(content, relief = 'groove', text='    Data processing tool    ', style='Yell.TLabel')

content.grid(column=0, row=0, sticky=(N, S, E, W))
frame_r.grid(column=1, row=1, columnspan=4, rowspan=5, sticky=(N, S, E, W))
frame_l.grid(column=0, row=1, columnspan=1, rowspan=5, sticky=(N, S, E, W))
namelbl.grid(column=0, row=0, columnspan=5, sticky=(N), padx=5)



# initialize Mongo connection
try:
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    #print('Connected to the MongoDB and found '+str(client.list_database_names()))
except pymongo.errors.ConnectionFailure as pymongo_error:
    messagebox.showwarning(title='Error', message='Error connecting to MongoDB: '+str(pymongo_error))
    window.destroy()
    exit(0)

# get history table
try:
    index = client.list_database_names().index('fsis')
    db=client['fsis']
    #print('Connected to fsis db')
except ValueError:
    messagebox.showwarning(title='Warning', message='Unable to find the db, new one will be created')

hist_collection = db['hist']

global ver
ver = ''

# create new record in history table

hist_record = {'index':int(time.time()),'state':'New'}
ins = hist_collection.insert_one(hist_record)
    
# set global variable ver to a new value 
ver = list(hist_record.values())[0]
#print ('ver value is',ver)
    
hist_df = pd.DataFrame(list(hist_collection.find()))

# match collection names with mongo 

collection1 = db['inventory'+str(ver)]
collection2 = db['inspections'+str(ver)]
collection3 = db['violations'+str(ver)]    


# drawing python logo

try:
    img_url = 'https://www.python.org/static/community_logos/python-powered-w-200x80.png'
    response = requests.get(img_url)
    img_data = response.content
    logo = ImageTk.PhotoImage(Image.open(BytesIO(img_data)))
except:
    print('There was a problem opening a logo file')
    logo = tk.PhotoImage(file='logo.png')
    
logo_label = tk.Label(frame_r, image=logo).grid(column=2, row=1, columnspan=4, sticky = (N))

    
# radiobuttons group section
v = tk.IntVar()
v.set(1)

    
# function for load button 

def load_btn():
    
    # drawing functional frame 
    
    # cleanup first 
    for widget in frame_r.winfo_children():
        widget.destroy()
    
   
    # empty list 
    file_list = {'inventory':'','inspections':'','violations':''}
    
    # promt user for each file in succession
    for x in file_list.keys():
        window.filename = filedialog.askopenfilename(initialdir = "/",title = 'Select file for '+x,filetypes = (("CSV files","*.csv"),("all files","*.*")))
        file_list[x] = window.filename
        #print (window.filename)
    
    # if at least one file is not selected, use all hardcoded values
    for y in file_list.keys():
        if file_list[y]=='':
            messagebox.showwarning(title='Error', message='Incomplete dataset selected, default ones will be used')
            file_list = {'inventory':'inventory.csv','inspections':'inspections.csv','violations':'violations.csv'}
        else:
            pass
    
    # attempt to read from files and re-encode them in UTF8
    
    start = time.time()
    for z in file_list.values():
        try:
            with io.open(z, 'r', encoding='utf-8-sig') as f:
                text = f.read()
            with io.open(z, 'w', encoding='utf-8', errors='ignore') as f:
                f.write(text)
                f.close()
        except FileNotFoundError:
            messagebox.showwarning(title='Error', message='There was a problem opening a file')
        except Exception as e:
            return(e)
    end = time.time()
    print('UTF conversion took',end - start)
    
    # let user know the status
    messagebox.showinfo(title='Info', message='Data files are now ready')
    
    # updating message
    label_load = tk.Label(frame_r, text='Data is loaded')
    label_load.grid(column=1, row=5, columnspan = 4, sticky=(S, W))
    
    # assiging list of files to global variable to pass to next button
    #print (file_list)
    global file_list_global
    file_list_global = file_list

    

# function for transform button 

def transform_btn():
    
    # drawing functional frame 
    # cleanup first 
    for widget in frame_r.winfo_children():
        widget.destroy()
    
    # split function
    def risk(df):
        return(' ').join(df.split(' ')[-2:])

    # prompt user for cleanup
    answer = messagebox.askyesno("Question","Perform data cleanup as well?")
    
    for in_file in file_list_global.values():
        
        out_file = (in_file.split(".")[0])+".json"
        
        # read csv again - now nicely formatted in utf-8
        csv_file = pd.DataFrame(pd.read_csv(in_file, sep = ",", engine = 'python', header = 0, index_col = False,encoding='utf-8'))

        # split PE DESCRIPTION and ACTIVITY DATE normalize date for inspections only
        if in_file.lower().find('inspections') != -1:
        
            #better split
            csv_file[['PE TYPE']] = csv_file['PE DESCRIPTION'].str.extract(r'(.+(?= \())',expand=True)
            csv_file[['SEATS']] = csv_file['PE DESCRIPTION'].str.extract(r'((?<=\().+(?=\)))',expand=True)
            csv_file[['PE RISK']] = csv_file['PE DESCRIPTION'].apply(risk)
        
            #date to MM DD and YYYY
            csv_file[['MM','DD','YYYY']] = csv_file['ACTIVITY DATE'].str.split("\/",expand=True)
        else:
            pass

        # normalize zip code to 5 digits
    
        def zap_zip(x):
            return x[:5]
                
        #if in_file != 'violations.csv':
        if in_file.lower().find('violations') == -1:
            csv_file['FACILITY ZIP'] = csv_file['FACILITY ZIP'].apply(zap_zip)
            csv_file['FACILITY ZIP'] = csv_file['FACILITY ZIP'].astype(str).astype(int)
        else: 
            pass
        
        # remove duplicates and replace empty with NaN
        
        if answer == True:
            csv_file.drop_duplicates()
            csv_file.replace(r'^\s+$', np.nan, regex=True)
    
        # check for existing file
        
        start = time.time()
        file_is_on = glob.glob(out_file)
        if not file_is_on:
            csv_file.to_json(out_file, orient = "records", date_format = "iso", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)
        else:
            answer = messagebox.askyesno(title='Conflict found',message='Json file '+out_file+' exists already. Overwrite?')
            if answer == True:
                csv_file.to_json(out_file, orient = "records", date_format = "iso", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)
            else:
                pass
        end = time.time()
        print('CSV to JSON took ',end - start)
        
    # let user know the status
    if answer == True:
        messagebox.showinfo(title='Info', message='Cleanup and transform to JSON is done')                
    else:
        messagebox.showinfo(title='Info', message='Transform to JSON is done')
    

    
    # get list of json files
    
    out_files_list = []
    for in_file in file_list_global.values():
        out_files_list.append((in_file.split(".")[0])+".json")
    #print(out_files_list)
    
    # load jsons to mongo 
    start = time.time()
    for file,collection in zip(out_files_list,[collection1,collection2,collection3]):
        with open(file, 'r') as file:
            temp_dict = json.load(file)
            try:
                collection.insert_many(temp_dict)
            except pymongo.errors.WriteError as pymongo_error:
                messagebox.showwarning(title='Error', message=pymongo_error)
    end = time.time()
    print('JSON to Mongo took ',end - start)
    
    # let user know the status
    messagebox.showinfo(title='Info', message='Loaded JSON files to the DB')                
                
    # update status of the working set

    query = {'index':ver}
    new_state = {'$set':{'state':'Data transformed'}}
    upd = hist_collection.update_one(query,new_state)    

    # updating message
    label_load = tk.Label(frame_r, text='Data is uploaded to the database')
    label_load.grid(column=1, row=5, columnspan = 4, rowspan = 5, sticky=(S, W))
    
# function for analyze button 

def analyze_btn():
    
    global file_list
    file_list = []
    
    # reset collection names
    collection1 = db['inventory'+str(ver)]
    collection2 = db['inspections'+str(ver)]
    collection3 = db['violations'+str(ver)]    
    collection4 = db['inspections_active'+str(ver)]
    
    # define mean, mode, median functions 
    
    def mmm_mean (field):
        mean = mmm_df.groupby([field,'YYYY'])['SCORE'].mean()
        return (mean)

    def mmm_mode (field):
        mode = mmm_df.groupby([field,'YYYY'])['SCORE'].apply(lambda x: x.mode().iloc[0])
        return (mode)

    def mmm_median (field):
        median = mmm_df.groupby([field,'YYYY'])['SCORE'].median()
        return (median)
    
    # function to create html
    
    def to_html(agg_function, by_field):
        
        html_out = agg_function(by_field).to_frame().round(1).to_html()
        o_file = open(agg_function.__name__+' '+by_field+' '+'.html', 'w')
        o_file.write(html_out)
        file_list.append(agg_function.__name__+' '+by_field+' '+'.html')
        o_file.close()
    
    # filter out active program status
    #print(ver)
    #print(collection2)
    active_query = {"PROGRAM STATUS": "ACTIVE"}
    mmm_df = pd.DataFrame(list(collection2.find(active_query)))
    if mmm_df.empty:
        messagebox.showwarning(title='Error', message='No data found, load and transform data first')
    else:
        
        # cast zip as int
        mmm_df['FACILITY ZIP'] = mmm_df['FACILITY ZIP'].astype(str).astype(int)
        try:
            collection4.insert_many(mmm_df.to_dict('records'))
            messagebox.showinfo(title='Info', message='Active only filter applied')
        except pymongo.errors.BulkWriteError as pymongo_error:
            messagebox.showwarning(title='Error', message='Data is already present, duplicates are not allowed')
    
    # create html files with mode, median and mean for venue type and zips
    
    start = time.time()
    
    to_html(mmm_mode,'PE TYPE')
    to_html(mmm_mean,'PE TYPE')
    to_html(mmm_median,'PE TYPE')
    to_html(mmm_mode,'FACILITY ZIP')
    to_html(mmm_mean,'FACILITY ZIP')
    to_html(mmm_median,'FACILITY ZIP')
    
    end = time.time()
    print('Aggregation to HTML took ',end - start)
    
    # drawing functional frame 
    
    # cleanup first 
    for widget in frame_r.winfo_children():
        widget.destroy()
    
    # function for opening local htmls
    def open_file():
        selected_file = str(var.get())
        webbrowser.open_new(selected_file)
    
    # dropdown list and buttons
    
    var = tk.StringVar(frame_r)
    var.set('')
    choices=file_list
    option = tk.OptionMenu(frame_r, var, *choices)
    btn_open_ok = ttk.Button(frame_r, text='View', command = open_file)

    option.grid(column=1, row=2, sticky=(N, W))
    btn_open_ok.grid(column=4, row=2, sticky=(S, E))

    # label    
    label_analyze = tk.Label(frame_r, text='Available files:')
    label_analyze.grid(column=1, row=1, columnspan = 4,sticky=(N))
    

# function for visualize button 

def visualize_btn():
    
    # reset collection names
    collection1 = db['inventory'+str(ver)]
    collection2 = db['inspections'+str(ver)]
    collection3 = db['violations'+str(ver)]    
    collection4 = db['inspections_active'+str(ver)]
    
    # drawing functional frame 
    
    # cleanup first 
    for widget in frame_r.winfo_children():
        widget.destroy()
    

    # barplot building function
    def show_fig():
    
        # load data to dataframe    
        #collection4 = db['inspections_active'+str(ver)]
        mmm_df = pd.DataFrame(list(collection4.find()))

        agg_type1_in = var1.get()
        agg_type2_in = var2.get()
        
        # check if input is present and correct
        if agg_type1_in == '' or agg_type2_in == '':
            messagebox.showwarning(title='Error', message='Incomplete filter selection')
            return
        
        # function to create a figure 
        def create_plot(agg_type1,agg_type2):

            sns.set(style="whitegrid")
            f, ax = plt.subplots(figsize=(10, 6))
            print(agg_type1, agg_type2)
            if agg_type2 == 'PE TYPE':
                if agg_type1 == 'MEDIAN':
                    sns.barplot(x = 'YYYY', y = 'SCORE', hue = 'PE TYPE', data = mmm_df, palette = 'rocket', estimator = np.median).set_title(agg_type1)
                elif agg_type1 == 'MEAN':
                    sns.barplot(x = 'YYYY', y = 'SCORE', hue = 'PE TYPE', data = mmm_df, palette = 'rocket', estimator = np.mean).set_title(agg_type1)
                else:
                    sns.barplot(x = 'YYYY', y = 'SCORE', hue = 'PE TYPE', data = mmm_df, palette = 'rocket').set_title(agg_type1)
            else:
                
                mmm_df['ZIP GROUP'] = pd.qcut(mmm_df['FACILITY ZIP'], 10)
                
                if agg_type1 == 'MEDIAN':
                    sns.barplot(x = 'YYYY', y = 'SCORE', hue = 'ZIP GROUP', data = mmm_df, palette = 'rocket', estimator = np.median).set_title(agg_type1)
                elif agg_type1 == 'MEAN':
                    sns.barplot(x = 'YYYY', y = 'SCORE', hue = 'ZIP GROUP', data = mmm_df, palette = 'rocket', estimator = np.mean).set_title(agg_type1)
                else:
                    sns.barplot(x = 'YYYY', y = 'SCORE', hue = 'ZIP GROUP', data = mmm_df, palette = 'rocket').set_title(agg_type1)
            return f

        # showing figure in a new window

        root = tk.Tk()
        root.wm_title('New window figure')

        label = tk.Label(root, text=str(agg_type1_in)+' for '+str(agg_type2_in))
        label.grid()

        fig = create_plot(agg_type1_in, agg_type2_in)
        canvas = FigureCanvasTkAgg(fig, master=root)
        canvas.draw()
        canvas.get_tk_widget().grid()    

        button_close = tk.Button(root, text="Close", command=root.destroy)
        button_close.grid()

        tk.mainloop()
    
    # function for reset button
    def reset_sel():
        var1.set('')
        var2.set('')
        
    # dataset
    active_query = {'VIOLATION  STATUS': 'OUT OF COMPLIANCE'}
    viol_df = pd.DataFrame(list(collection3.find(active_query)))
    if viol_df.empty:
        messagebox.showwarning(title='Error', message='Nothing to visualize')
        return
        
    # estimate if dataset size is suitable for visualization 
        
    feat_len = len(viol_df['VIOLATION CODE'].unique())
    if feat_len > 50:
        viol_df = viol_df[~viol_df['VIOLATION CODE'].isin(viol_df['VIOLATION CODE'].value_counts().nsmallest(20).index)]
        print('Dataset was trimmed due its size')
    else:
        pass
    
    # group and create matrix, merge with inspections for zip 
    
    inspections_df = pd.DataFrame(list(collection4.find()))
    merged_viol = pd.merge(left=viol_df, right=inspections_df, how='left', left_on='SERIAL NUMBER', right_on='SERIAL NUMBER')
    merged_viol = merged_viol[['VIOLATION CODE','POINTS','FACILITY ID', 'FACILITY ZIP','PE TYPE','SEATS','ACTIVITY DATE', 'MM','DD','YYYY']].dropna()
    
    # group and count violation code occurrences 
    
    grouped_viol_zip = merged_viol.groupby(['FACILITY ZIP','VIOLATION CODE'])
    num_viol_zip = grouped_viol_zip.agg({'VIOLATION CODE': 'count'})
    
  
    
    viol_mx_zip = num_viol_zip['VIOLATION CODE'].unstack(0).T
    viol_mx_zip.fillna(0, inplace=True)
    viol_mx_zip.reset_index(inplace=True)
    
    # casting zip as int for correlation
    #viol_mx_zip['FACILITY ZIP'] = viol_mx_zip['FACILITY ZIP'].astype(str).astype(int)
    
    # pearson correlation matrix 
    tmp_corr = viol_mx_zip.corr()       
    
    # function for violations scatterplot 
    def show_scatter():
        
        # creating a plot  
        def built_scatter():
            
            print('Building countplot')
            start = time.time()
            
            f, ax = plt.subplots(figsize=(20, 10))
            if merged_viol['VIOLATION CODE'].value_counts().count() > 20:
                sns.countplot(x = 'VIOLATION CODE', data=merged_viol, palette = 'rocket', order=merged_viol['VIOLATION CODE'].value_counts().iloc[:20].index).set_title('Top 20 violations')
                print('Too many features for countplot, scaling down to 20')
            else:
                sns.countplot(x = 'VIOLATION CODE', data=merged_viol, palette = 'rocket', order=merged_viol['VIOLATION CODE'].value_counts().index).set_title('Count of violations')
            
            end = time.time()
            print('Building count plot took ',end - start)
            
            return f


        
        # displaying in a new window
        
        root = tk.Tk()
        root.wm_title('Violations countplot')

        label = tk.Label(root, text='New window figure')
        label.grid()
        
        sc_plot = built_scatter()
        canvas = FigureCanvasTkAgg(sc_plot, master=root)
        canvas.draw()
        canvas.get_tk_widget().grid()    

        button_close = tk.Button(root, text="Close", command=root.destroy)
        button_close.grid()

        tk.mainloop()
        
    # function for zip vs violations correlation matrix 
    
    def show_matrix():

            # creating matrix plot
        def built_matrix():  
                    
            print('Building correlation matrix')
            start = time.time()
            
            sns.set(style='whitegrid')
            f, ax = plt.subplots(figsize=(20, 10))
            cmap = sns.cm.rocket_r
            threshold_value = tmp_corr[(tmp_corr > 0.6)]
            sns.heatmap(threshold_value)
            
            end = time.time()
            print('Building correlation matrix took ',end - start)
            
            return f

        # displaying in a new window
        
        root = tk.Tk()
        root.wm_title('Correlation matrix')

        label = tk.Label(root, text='New window figure')
        label.grid()
        
        sc_plot = built_matrix()
        canvas = FigureCanvasTkAgg(sc_plot, master=root)
        canvas.draw()
        canvas.get_tk_widget().grid()    

        button_close = tk.Button(root, text="Close", command=root.destroy)
        button_close.grid()

        tk.mainloop()
        
        
    
    # dropdowns for filtering options and button to show the plot 
    agg_type1_list = ['MEDIAN', 'MODE', 'MEAN']
    agg_type2_list = ['PE TYPE', 'FACILITY ZIP']
    
    var1 = tk.StringVar(frame_r)
    var1.set('')
    choices1=agg_type1_list
    option1 = tk.OptionMenu(frame_r, var1, *choices1)
    
    var2 = tk.StringVar(frame_r)
    var2.set('')
    choices2=agg_type2_list
    option2 = tk.OptionMenu(frame_r, var2, *choices2)
    
    btn_show_graph = ttk.Button(frame_r, text='Create plot', command = show_fig)
    btn_reset = ttk.Button(frame_r, text='Reset', command = reset_sel)
    label1_agg = tk.Label(frame_r, text='Aggregation')
    label2_agg = tk.Label(frame_r, text='Group')
    
    # violations scatterplot button 
    btn_show_scatter = ttk.Button(frame_r, text='Violations plot', command = show_scatter)
    
    
    # correlation matrix button 
    btn_show_matrix = ttk.Button(frame_r, text='Correlation matrix', command = show_matrix)
    
       
    label1_agg.grid(column=1, row=1, sticky=(W))
    option1.grid(column=1, row=2, sticky=(W))
    
    label2_agg.grid(column=1, row=3, sticky=(W))
    option2.grid(column=1, row=4, sticky=(W))
    
    btn_show_graph.grid(column=1, row=5, sticky=(W))
    btn_reset.grid(column=2, row=5, sticky=(W))
    
    btn_show_scatter.grid(column=3, row=2, sticky=(W))
    btn_show_matrix.grid(column=3, row=4, sticky=(W))
    


# function for restore button 
    
def restore_btn():
    #print(hist_df)
    
    
    def restored():
        global ver
        ver = var.get()
        messagebox.showinfo(title='Info', message='Restored to '+str(ver))

    # drawing functional frame 
    
    # cleanup first 
    for widget in frame_r.winfo_children():
        widget.destroy()
    

    var = tk.StringVar(frame_r)
    var.set('')
    choices=hist_df['index']
    option = tk.OptionMenu(frame_r, var, *choices)
    btn_restore_ok = ttk.Button(frame_r, text='Restore', command = restored)
    label_restore = tk.Label(frame_r, text='Available points to restore')
    label_text_str = 'Current dataset version is '+str(ver)
    label_ver = tk.Label(frame_r, text=label_text_str)
    
    label_restore.grid(column=1, row=1, columnspan = 4, sticky=(N))
    option.grid(column=1, row=2, sticky=(N, W))
    btn_restore_ok.grid(column=4, row=2, sticky=(S, E))
    label_ver.grid(column=1, row=3, columnspan = 4, sticky=(N,W))


    

        
# creating radio buttons group
    
r_load = tk.Radiobutton(frame_l, indicatoron = 0, text='Load new', variable=v, command=load_btn, value=101)
r_load.grid(row=1, column=0, sticky=(W, E, N))

r_trans = tk.Radiobutton(frame_l, indicatoron = 0, text='Transform', variable=v, command=transform_btn, value=102)
r_trans.grid(row=2, column=0, sticky=(W, E, N))

r_an = tk.Radiobutton(frame_l, indicatoron = 0, text='Analyze', variable=v, command=analyze_btn, value=103)
r_an.grid(row=3, column=0, sticky=(W, E, N))

r_vis = tk.Radiobutton(frame_l, indicatoron = 0, text='Visualize', variable=v, command=visualize_btn, value=104)
r_vis.grid(row=4, column=0, sticky=(W, E, N))

r_restore = tk.Radiobutton(frame_l, indicatoron = 0, text='Restore', variable=v, command=restore_btn, value=105)
r_restore.grid(row=5, column=0, sticky=(W, E, N))

    
# generic buttons functionality

# function for opening htmls
def open_url(url):
    webbrowser.open_new(url)

## help file url and function
url = 'https://github.com/olegph/DataProcessingTool/blob/main/README.md'

## exit button function
def exit_button_click():
    answer = messagebox.askyesno("Question","Close application?")
    if answer is True:
        window.destroy()
    else:
        pass
        
# buttons
btn_help = ttk.Button(content, text='Help', command = lambda aurl=url:open_url(aurl))
btn_exit = ttk.Button(content, text='Exit', command = exit_button_click)

btn_help.grid(column=4, row=7, sticky=(S, E))
btn_exit.grid(column=0, row=7, sticky=(S, W))


# sizing and positioning 
window.columnconfigure(0, weight=1)
window.rowconfigure(0, weight=1)

for i in range(5):
    content.columnconfigure(i, weight=3)

for j in range(7):
    content.rowconfigure(j, weight=1)
    
window.mainloop()

There was a problem opening a logo file
